In [11]:
import os

MODEL_ROOT = os.path.join(os.getcwd(), 'models')

constant = {
    'faceModel': os.path.join(MODEL_ROOT, 'res10_300x300_ssd_iter_140000.caffemodel'),
    'faceProto': os.path.join(MODEL_ROOT, 'deploy.prototxt'),
    'ageModel': os.path.join(MODEL_ROOT, 'age_net.caffemodel'),
    'ageProto': os.path.join(MODEL_ROOT, 'age_deploy.prototxt'),
    'genderModel': os.path.join(MODEL_ROOT, 'gender_net.caffemodel'),
    'genderProto': os.path.join(MODEL_ROOT, 'gender_deploy.prototxt')
}

age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
gender_list = ['Male', 'Female']

In [13]:
import numpy as np
import cv2


# Load attribute models
face_proto = constant['faceProto']
face_model = constant['faceModel']

age_proto = constant['ageProto']
age_model = constant['ageModel']

gender_proto = constant['genderProto']
gender_model = constant['genderModel']

face_net = cv2.dnn.readNetFromCaffe(face_proto, face_model)
age_net = cv2.dnn.readNetFromCaffe(age_proto, age_model)
gender_net = cv2.dnn.readNetFromCaffe(gender_proto, gender_model)

MODEL_MEAN_VALUES = MODEL_MEAN_VALUES
gender_list = gender_list
age_list = age_list

padding = 20


def get_face(frame, conf_threshold=0.75):
    frame_opencv_dnn = frame.copy()
    frame_height, frame_width, _ = frame_opencv_dnn.shape

    # optimized_img = cv2.medianBlur(frame_opencv_dnn, 5)
    #
    # cv2.imshow('op', frame_opencv_dnn)

    gender = None
    age = None

    try:
        blob = cv2.dnn.blobFromImage(frame_opencv_dnn, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        # face_boxes = {}

        face_net.setInput(blob)
        detections = face_net.forward()

        # gender_net.setInput(blob)
        # gender_prediction = gender_net.forward()
        # gender = gender_list[gender_prediction[0].argmax()]
        #
        # age_net.setInput(blob)
        # age_prediction = age_net.forward()
        # age = age_list[age_prediction[0].argmax()]

        confidences = detections[0, 0, :, 2]
        indices = np.where(confidences >= conf_threshold)[0]

        # indices = np.where(confidences > conf_threshold)[0]
        #
        boxes = detections[0, 0, indices, 3:7] * np.array([frame_width, frame_height, frame_width, frame_height])
        boxes = boxes.astype(int)
        #
        confidences_filtered = confidences[indices]
        #
        face_boxes = {confidence: box.tolist() for confidence, box in zip(confidences_filtered, boxes)}


        # for i in indices:
        #     box = detections[0, 0, i, 3:7] * np.array([frame_width, frame_height, frame_width, frame_height])
        #     x1, y1, x2, y2 = box.astype(int)
        #     confidence = confidences[i]
        #     face_boxes[confidence] = [x1, y1, x2, y2]

            # x1 = int(detections[0, 0, i, 3] * frame_width)
            # y1 = int(detections[0, 0, i, 4] * frame_height)
            # x2 = int(detections[0, 0, i, 5] * frame_width)
            # y2 = int(detections[0, 0, i, 6] * frame_height)
            #
            # confidence = confidences[i]
            # face_boxes[confidence] = [x1, y1, x2, y2]

        if face_boxes:
            max_confidence = max(face_boxes.keys())
            gender, age = predict_attribute(face=face_boxes[max_confidence], frame=frame)
        return [gender, age]

        # for i in range(detections.shape[2]):
        #     confidence = detections[0, 0, i, 2]
        #     if confidence > conf_threshold:
        #         x1 = int(detections[0, 0, i, 3] * frame_width)
        #         y1 = int(detections[0, 0, i, 4] * frame_height)
        #         x2 = int(detections[0, 0, i, 5] * frame_width)
        #         y2 = int(detections[0, 0, i, 6] * frame_height)
        #
        #         face_boxes[confidence] = [x1, y1, x2, y2]
        #
        # if face_boxes:
        #     gender, age = predict_attribute(face=face_boxes[max(face_boxes.keys())], frame=frame)
        #     return [gender, age]
    except Exception:
        return []


def predict_attribute(face, frame):
    """
    Predict gender and age based on the detected face region in the frame.

    Args:
    face (list): List containing coordinates [x1, y1, x2, y2] of the detected face.
    frame (numpy.ndarray): Frame containing the detected face region.

    Returns:
    tuple: A tuple containing predicted gender and age based on the detected face.
    (gender, age) will be returned. If face_img is empty or an error occurs during prediction,
    'undefined' will be returned for both gender and age.

    This function predicts the gender and age of a detected face using pre-trained models.
    It extracts the face region from the frame and performs inference using gender_net and age_net.
    """
    face_img = frame[max(0, face[1] - padding): min(face[3] + padding, frame.shape[0] - 1), max(0, face[0] - padding):min(face[2] + padding, frame.shape[1] - 1)]

    gender = None
    age = None

    if len(face_img) != 0:
        blob2 = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        gender_net.setInput(blob2)
        gender_prediction = gender_net.forward()
        gender = gender_list[gender_prediction[0].argmax()]

        age_net.setInput(blob2)
        age_prediction = age_net.forward()
        age = age_list[age_prediction[0].argmax()]

    return gender, age


def make_shadow(im0, coords):
    """
    Create a shadow effect on an image based on the provided coordinates.

    Args:
    im0 (numpy.ndarray): The input image.
    coords (list): List of coordinates defining the shape for which the shadow will be created.

    Returns:
    numpy.ndarray: An image with the applied shadow effect.
    """
    # Step 1: Make a copy of that image
    shadow = im0.copy()

    # Step 2: Create a mask for the filled shape
    mask = np.zeros_like(im0)  # Create a black mask with the same size as the image
    cv2.fillPoly(mask, [coords], (255, 255, 255))  # Fill the shape with white in the mask

    # Step 3: Create a transparent shadow image
    shadow_alpha = 0.5  # Adjust transparency here (0.0 to 1.0)
    shadow[:, :, 3] = shadow_alpha * 255

    # Step 4: Overlay the shadow on the original image using the mask
    result = cv2.add(im0, shadow, mask=mask)

    return result


In [15]:
video=cv2.VideoCapture(2)
padding=20

while True:
    ret,frame=video.read()
    frame,bboxs=get_face(frame)
    for bbox in bboxs:
        # face=frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
        face = frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding, frame.shape[1]-1)]
        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        gender_model.setInput(blob)
        genderPred=gender_model.forward()
        gender=gender_list[genderPred[0].argmax()]


        age_model.setInput(blob)
        agePred=age_model.forward()
        age=age_list[agePred[0].argmax()]


        label="{},{}".format(gender,age)
        cv2.rectangle(frame,(bbox[0], bbox[1]-30), (bbox[2], bbox[1]), (0,255,0),-1) 
        cv2.putText(frame, label, (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2,cv2.LINE_AA)
    cv2.imshow("Age-Gender",frame)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break
video.release()
cv2.destroyAllWindows()

[ WARN:0@493.295] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@493.413] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


AttributeError: 'NoneType' object has no attribute 'copy'